[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openlayer-ai/examples-gallery/blob/main/text-classification/documentation-tutorial/nlp-tutorial-part-4.ipynb)



# <a id="top">Openlayer text classification tutorial - Part 4</a>

Welcome! This is the final notebook from the text classification tutorial. Here, we solve the **subpopulation performance** issue and commit the new datasets and model versions to the platform. You should use this notebook together with the **text classification tutorial from our documentation**.


## <a id="toc">Table of contents</a>

1. [**Fixing the performance issue and re-training the model**](#1)
    

2. [**Using Openlayer's Python API**](#2)

In [ ]:
%%bash

if [ ! -e "requirements.txt" ]; then
    curl "https://raw.githubusercontent.com/openlayer-ai/examples-gallery/main/text-classification/documentation-tutorial/requirements.txt" --output "requirements.txt"
fi

In [ ]:
!pip install -r requirements.txt

## <a id="1"> 1. Fixing the data integrity issues and re-training the model </a>

[Back to top](#top)

In this first part, we will download a new version of the training set. This new version was obtained by augmenting the original training set to ensure there are more rows mentioning the canadian hockey teams `"Canadiens"` and `"Senators`", which we've found to be problematic for the model.

We obtained the synthetic data samples by perturbing the original training set. If a row mentioned one of the teams, we also added a new row mentioning the other. For example, if the row `"The Canadiens are doing alright this season"`, we add another row with `"The Senators are doing alright this season"`.

We kept the validation set the same.

In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

### <a id="download">Downloading the dataset </a>

In [2]:
%%bash

if [ ! -e "20_news_train_performance_fix.csv" ]; then
    curl "https://openlayer-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/text-classification/documentation/20_news_train_performance_fix.csv" --output "20_news_train_performance_fix.csv"
fi

if [ ! -e "20_news_val_consistency_fix.csv" ]; then
    curl "https://openlayer-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/text-classification/documentation/20_news_val_consistency_fix.csv" --output "20_news_val_consistency_fix.csv"
fi

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  933k  100  933k    0     0   256k      0  0:00:03  0:00:03 --:--:--  256k


In [3]:
# Loading and having a look at the training set
training_set = pd.read_csv("./20_news_train_performance_fix.csv", index_col=0)
validation_set = pd.read_csv("./20_news_val_consistency_fix.csv")

training_set.head()

,text,label
0,Ditto... If we allow people like him to conti...,hockey
1,Babe Ruth's lifetime pitching stats (selected)...,baseball
2,1993 CALDER CUP PLAYOFF SCHEDULE AND RESULTS ...,hockey
3,"""Deeply rooted rivalry?"" Ahem, Jokerit have be...",hockey
4,If it encouraged the runner to stretch his lea...,baseball


### <a id="train">Training the model</a>

In [4]:
sklearn_model = Pipeline([('count_vect', CountVectorizer(ngram_range=(1,2), stop_words='english')), 
                          ('lr', GradientBoostingClassifier(n_estimators=300, random_state=42))])
sklearn_model.fit(training_set['text'], training_set['label'])

Pipeline(steps=[('count_vect',
                 CountVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('lr',
                 GradientBoostingClassifier(n_estimators=300,
                                            random_state=42))])

In [5]:
print(classification_report(validation_set['label'], sklearn_model.predict(validation_set['text'])))

              precision    recall  f1-score   support

    baseball       0.85      0.98      0.91       183
      hockey       0.98      0.84      0.90       201

    accuracy                           0.91       384
   macro avg       0.91      0.91      0.91       384
weighted avg       0.92      0.91      0.91       384



## <a id="2"> 2. Using Openlayer's Python API</a>

[Back to top](#top)

Now it's time to upload the datasets and model to the Openlayer platform.

In [ ]:
!pip install openlayer

### <a id="client">Instantiating the client</a>

In [6]:
import openlayer

client = openlayer.OpenlayerClient("YOUR_API_KEY_HERE")

### <a id="project">Loading a project from the platform</a>

In [7]:
from openlayer.tasks import TaskType

project = client.create_or_load_project(
    name="Hockey x Baseball - 20 newsgroups",
    task_type=TaskType.TextClassification,
    description="Evaluation of ML approaches for text classification"
)

Found your project. Navigate to http://localhost:8000/tutorials/77aa09dd-1fc9-4f44-9c91-eedd61c76ff7 to see it.


### <a id="dataset">Uploading datasets</a>

In terms of configuration, the datasets haven't changed from the previous version. For completeness, we will write new `dataset_config.yaml` files.

First, let's start by enhancing the datasets with the extra column:

In [8]:
# Adding the column with the predictions (since we'll also upload a model later)
training_set["predictions"] = sklearn_model.predict_proba(training_set["text"]).tolist()
validation_set["predictions"] = sklearn_model.predict_proba(validation_set["text"]).tolist()

In [9]:
string_to_int_map = {"baseball": 0, "hockey": 1}
training_set["label"] = training_set["label"].map(string_to_int_map)
validation_set["label"] = validation_set["label"].map(string_to_int_map)

In [10]:
# Some variables that will go into the `dataset_config.yaml` file
class_names = ["Baseball", "Hockey"]
column_names = list(training_set.columns)
text_column_name = "text"
label_column_name = "label"
predictions_column_name = "predictions"

In [11]:
import yaml 

# Note the camelCase for the dict's keys
training_dataset_config = {
    "classNames": class_names,
    "columnNames": column_names,
    "textColumnName": "text",
    "label": "training",
    "labelColumnName": label_column_name,
    "predictionsColumnName": predictions_column_name,
}

with open("training_dataset_config.yaml", "w") as dataset_config_file:
    yaml.dump(training_dataset_config, dataset_config_file, default_flow_style=False)

In [12]:
import copy

validation_dataset_config = copy.deepcopy(training_dataset_config)

# In our case, the only field that changes is the `label`, from "training" -> "validation"
validation_dataset_config["label"] = "validation"

with open("validation_dataset_config.yaml", "w") as dataset_config_file:
    yaml.dump(validation_dataset_config, dataset_config_file, default_flow_style=False)

In [13]:
# Training set
project.add_dataframe(
    dataset_df=training_set,
    dataset_config_file_path="training_dataset_config.yaml",
)

Staged the `training` resource!


In [14]:
# Validation set
project.add_dataframe(
    dataset_df=validation_set,
    dataset_config_file_path="validation_dataset_config.yaml",
)

Staged the `validation` resource!


We can check that both datasets are now staged using the `project.status()` method. 

In [15]:
project.status()

The following resources are staged, waiting to be committed:
	 - training
	 - validation
Use the `commit` method to add a commit message to your changes.


### <a id="model">Uploading models</a>

Once we're done with the consistency goals, we'll move on to performance goals, which have to do with the model itself. Therefore, now, we will upload a **full model** instead of a shell model. We will do so so that we can have explain the model's predictions on the platform using explainability techiques such as LIME.

#### <a id="full-model"> Full models </a>

To upload a full model to Openlayer, you will need to create a **model package**, which is nothing more than a folder with all the necessary information to run inference with the model. The package should include the following:
1. A `requirements.txt` file listing the dependencies for the model.
2. Serialized model files, such as model weights, encoders, etc., in a format specific to the framework used for training (e.g. `.pkl` for sklearn, `.pb` for TensorFlow, and so on.)
3. A `prediction_interface.py` file that acts as a wrapper for the model and implements the `predict_proba` function. 

Other than the model package, a `model_config.yaml` file is needed, with information about the model to the Openlayer platform, such as the framework used, feature names, and categorical feature names.

Lets prepare the model package one piece at a time.

In [16]:
# Creating the model package folder (we'll call it `model_package`)
!mkdir model_package

mkdir: model_package: File exists


**1. Adding the `requirements.txt` to the model package**

In [17]:
!scp requirements.txt model_package

**2. Serializing the model**

In [18]:
import pickle 

# Trained model
with open("model_package/model.pkl", "wb") as handle:
    pickle.dump(sklearn_model, handle, protocol=pickle.HIGHEST_PROTOCOL)

**3. Writing the `prediction_interface.py` file**

In [19]:
%%writefile model_package/prediction_interface.py

import pickle
from pathlib import Path

import pandas as pd

PACKAGE_PATH = Path(__file__).parent


class SklearnModel:
    def __init__(self):
        """This is where the serialized objects needed should
        be loaded as class attributes."""

        with open(PACKAGE_PATH / "model.pkl", "rb") as model_file:
            self.model = pickle.load(model_file)

    def predict_proba(self, input_data_df: pd.DataFrame):
        """Makes predictions with the model. Returns the class probabilities."""
        text_column = input_data_df.columns[0]
        return self.model.predict_proba(input_data_df[text_column])


def load_model():
    """Function that returns the wrapped model object."""
    return SklearnModel()

Overwriting model_package/prediction_interface.py


**Creating the `model_config.yaml`**

In [20]:
import yaml

model_config = {
    "name": "News classifier",
    "architectureType": "sklearn",
    "metadata": {  # Can add anything here, as long as it is a dict
        "model_type": "Gradient Boosting Classifier",
        "regularization": "None",
        "vectorizer": "Count Vectorizer"
    },
    "classNames": class_names,
}

with open("model_config.yaml", "w") as model_config_file:
    yaml.dump(model_config, model_config_file, default_flow_style=False)

Lets check that the model package contains everything needed:

In [21]:
from openlayer.validators import ModelValidator

model_validator = ModelValidator(
    model_package_dir="model_package", 
    model_config_file_path="model_config.yaml",
    sample_data = validation_set.iloc[:10, :],
)
model_validator.validate()

/Users/gustavocid/Desktop/openlayer-repos/openlayer-python-client/openlayer/validators.py:1237: Warning: There is a version discrepancy between the current environment and the dependency `pandas==1.1.4`. 
`requirements.txt` specifies `pandas==1.1.4`, but the current environment contains `pandas 1.5.3` installed. 
There might be unexpected results once the model is in the platform. Use at your own discretion.
  warnings.warn(


[]

All validations are passing, so we are ready to add the full model!

In [22]:
project.add_model(
    model_package_dir="model_package",
    model_config_file_path="model_config.yaml",
    sample_data=validation_set.iloc[:10, :],
)

Staged the `model` resource!


/Users/gustavocid/Desktop/openlayer-repos/openlayer-python-client/openlayer/validators.py:1237: Warning: There is a version discrepancy between the current environment and the dependency `pandas==1.1.4`. 
`requirements.txt` specifies `pandas==1.1.4`, but the current environment contains `pandas 1.5.3` installed. 
There might be unexpected results once the model is in the platform. Use at your own discretion.
  warnings.warn(


We can check that both datasets and model are staged using the `project.status()` method.

In [23]:
project.status()

The following resources are staged, waiting to be committed:
	 - training
	 - model
	 - validation
Use the `commit` method to add a commit message to your changes.


### <a id="commit"> Committing and pushing to the platform </a>

Finally, we can commit the new project version to the platform. 

In [27]:
project.commit("Augment training set by perturbing rows with Canadian teams and increase n_estimators")

There is nothing staged to commit. Please add model and/or datasets first before committing.


In [25]:
project.status()

The following resources are committed, waiting to be pushed:
	 - training
	 - model
	 - validation
Commit message from Sun Apr  2 13:07:31 2023:
	 Augment training set by perturbing rows with Canadian teams
Use the `push` method to push your changes to the platform.


In [26]:
project.push()

/Users/gustavocid/Desktop/openlayer-repos/openlayer-python-client/openlayer/validators.py:1237: Warning: There is a version discrepancy between the current environment and the dependency `pandas==1.1.4`. 
`requirements.txt` specifies `pandas==1.1.4`, but the current environment contains `pandas 1.5.3` installed. 
There might be unexpected results once the model is in the platform. Use at your own discretion.
  warnings.warn(


Pushing changes to the platform with the commit message: 
	 - Message: Augment training set by perturbing rows with Canadian teams 
	 - Date: Sun Apr  2 13:07:31 2023
Pushed!
